## canopy cover damage with windspeed
notebook to graph windspeed category with area of canopy cover affected

- load mangrove before and after canopy classes
- make before canopy classes 10, 20, 30, 2550
- add together
- xr.where to get areas only for canopy cover loss for each class (i.e. pixels of closed forest that were lost)
    damage to closed forest (values of 30, 29, 28, -225)
    damage to open forest (values of 20, 19, -235)
    damage to woodland (values of 10, -245)
- intersect with windspeed (based on xr.where statements - pretty hacky atm but gets the expected results
- final output csv table is messy, but makes sure cyclone wind categories and mangrove damage categories are clear


In [ ]:
import datacube
import numpy as np
import xarray as xr
import pandas as pd
import os
from datacube import Datacube
from datacube.utils import geometry
import rioxarray
from datacube.utils.cog import write_cog
import rioxarray
from datacube.testutils.io import rio_slurp_xarray

dc = datacube.Datacube(app="cyclone mangroves")

pd.set_option('display.max_columns', None)

### fill in data (doing one cyclone at a time for the moment)
option to export area of canopy cover classes as csv

In [ ]:
# cyclone_name = 'Yasi'
# cyclone_time = 2011

# cyclone_name = 'Larry'
# cyclone_time = 2006

# cyclone_name = 'Ita'
# cyclone_time = 2014

# cyclone_name = 'Marcia'
# cyclone_time = 2015

# cyclone_name = 'Lam'
# cyclone_time = 2015

# cyclone_name = 'George'
# cyclone_time = 2007

# cyclone_name = 'Ingrid_Landfall1'
# cyclone_time = 2005

# cyclone_name = 'Ingrid_Landfall2'
# cyclone_time = 2005

# cyclone_name = 'Ingrid_Landfall3'
# cyclone_time = 2005

# cyclone_name = 'Monica_Landfall1'
# cyclone_time = 2006

# cyclone_name = 'Monica_Landfall2'
# cyclone_time = 2006

# cyclone_name = 'Nathan_Landfall1'
# cyclone_time = 2015

# cyclone_name = 'Nathan_Landfall2'
# cyclone_time = 2015

cyclone_name = 'Laurence'
cyclone_time = 2009

export_as_csv = True

### load in geotiff

In [ ]:
geotiff_path = './FINALlocalWind/'+cyclone_name+'_30m_EPSG3577.tif'
# Open into an xarray.DataArray
geotiff_da = rioxarray.open_rasterio(geotiff_path)
# change -9999 to 0 #### not sure this needs to be done, -9999 should be considered 'not cyclone'?
geotiff_da = xr.where(geotiff_da == -9999, 0, geotiff_da.values)
# Covert our xarray.DataArray into a xarray.Dataset
geotiff_ds = geotiff_da.to_dataset('band')
# Rename the variable to a more useful name
dataset = geotiff_ds.rename({1: 'wind_speed'})

### load mangrove data on extent of geotiff

In [ ]:
# get extent of cyclone dataset
geobox = dataset.extent

In [ ]:
# before cyclone
time_before_cyclone = (str(cyclone_time-1) + '-01-01', str(cyclone_time-1) + '-12-31')

bc_mangrove = dc.load(product = 'ga_ls_mangrove_cover_cyear_3', 
                            geopolygon=dataset.extent, 
                            output_crs='EPSG:3577', 
                            time = time_before_cyclone,
                            resolution=(-30, 30))

time_after_cyclone = (str(cyclone_time) + '-01-01', str(cyclone_time) + '-12-31')

# after cyclone
ac_mangrove = dc.load(product = 'ga_ls_mangrove_cover_cyear_3', 
                            geopolygon=dataset.extent, 
                            output_crs='EPSG:3577', 
                            time = time_after_cyclone,
                            resolution=(-30, 30))

### get mangrove damage pixels

In [ ]:
# change bc_mangrove from 0,1,2,3,255 --> 0,10,20,30,2550
bc_mangrove_times10 = bc_mangrove.astype(dtype='int16') * 10
# check output is correct
# np.unique(bc_mangrove_times10.canopy_cover_class)

# change ac_mangrove to int16 as well
ac_mangrove_int = ac_mangrove.astype(dtype='int16')

# add together, remove time dim
mangrove_added = bc_mangrove_times10.canopy_cover_class.squeeze('time') - ac_mangrove_int.canopy_cover_class.squeeze('time')

In [ ]:
# get only damage values
mangrove_damage = xr.where((mangrove_added == -245) | (mangrove_added == 10) | # damage to woodland
                           (mangrove_added == -235) | (mangrove_added == 20) | (mangrove_added == 19) | # damage to open forest
                           (mangrove_added == -225) | (mangrove_added == 30) | (mangrove_added == 29) | (mangrove_added == 28), # damage to closed forest
                           mangrove_added.values, 0)

np.unique(mangrove_damage.values)

In [ ]:
# successive where statements to get damage categories back to simple 1,2,3 categories
woodland_damage = xr.where((mangrove_damage == -245) | (mangrove_damage == 10), 1, mangrove_damage.values)
openforest_damage = xr.where((woodland_damage == -235) | (woodland_damage == 20) | (woodland_damage == 19), 2, woodland_damage.values)
closedforest_damage = xr.where((openforest_damage == -225) | (openforest_damage == 30) | (openforest_damage == 29) | (openforest_damage == 28), 3, openforest_damage.values)

total_mangrove_damage = closedforest_damage.astype('uint8')
del woodland_damage, openforest_damage, closedforest_damage

# check it worked as expected
np.unique(total_mangrove_damage.values)

### rework to get geotiff as identical dc.load for mangroves

In [ ]:
# get geometry box from geotiff
full_box = geometry.GeoBox.from_geopolygon(geobox, resolution=(30, -30))

# load in geotiff again but with identical extent from dc.load mangroves
cyclone_da = rio_slurp_xarray(geotiff_path, gbox=ac_mangrove.geobox)
# change -9999 to 0 #### not sure this needs to be done, -9999 should be considered 'not cyclone'?
# cyclone_da = xr.where(cyclone_da == -9999, 0, cyclone_da.values)
# add time dimension
cyclone_da_time = cyclone_da.expand_dims(dim={"time": ac_mangrove.time})
# Covert our xarray.DataArray into a xarray.Dataset
cyclone_ds = cyclone_da_time.to_dataset(name="wind_speed")

# copy over attributes
cyclone_ds.attrs['grid_mapping'] = ac_mangrove.attrs['grid_mapping']
cyclone_ds.attrs['crs'] = ac_mangrove.attrs['crs']

In [ ]:
windspeed_category = {'C1': [0., 125*1000/60**2],
                        'C2':[125*1000/60**2, 165*1000/60**2],
                        'C3': [165*1000/60**2, 225*1000/60**2], 
                        'C4': [225*1000/60**2, 280*1000/60**2],
                        'C5': [280*1000/60**2, 9999.]}
windspeed_category

In [ ]:
cyclone_windspeed = cyclone_ds.wind_speed.squeeze('time')

In [ ]:
# get cyclone dataset into windspeed categories, successive where statements like above
cyclone_ds_c1 = xr.where((cyclone_windspeed >= 0.0) & (cyclone_windspeed < 34.72222222222222), 1, cyclone_windspeed.values)
cyclone_ds_c2 = xr.where((cyclone_ds_c1 >= 34.72222222222222) & (cyclone_ds_c1 < 45.833333333333336), 2, cyclone_ds_c1.values)
cyclone_ds_c3 = xr.where((cyclone_ds_c2 >= 45.833333333333336) & (cyclone_ds_c2 < 62.5), 3, cyclone_ds_c2.values)
cyclone_ds_c4 = xr.where((cyclone_ds_c3 >= 62.5) & (cyclone_ds_c3 < 77.77777777777777), 4, cyclone_ds_c3.values)
cyclone_ds_c5 = xr.where((cyclone_ds_c4 >= 77.77777777777777) & (cyclone_ds_c4 < 9999.0), 5, cyclone_ds_c4.values)

cyclone_categories = cyclone_ds_c5
del cyclone_ds_c1, cyclone_ds_c2, cyclone_ds_c3, cyclone_ds_c4, cyclone_ds_c5

# check it worked as expected
np.unique(cyclone_categories)

### get mangrove damage area relative to wind category

In [ ]:
# create dataframe to append to
df = pd.DataFrame()

# for damage canopy cover class (0,1,2,3)
for c in np.unique(total_mangrove_damage):
    category = cyclone_categories.where(total_mangrove_damage == c)
    unique, counts = np.unique(category, return_counts=True)
    item = np.asarray((unique, counts)).T
    # this next line makes the table messy, but couldn't find a better way to make sure 
    # area values alligned with unique cyclone wind category
    df['class '+str(c)+' unique values'] = pd.Series(unique)
    # append to df (divide counts by 0.0009 to give area in km2)
    df['class '+str(c)+ ' area (km2)'] = pd.Series(counts*0.0009)

In [ ]:
df

In [ ]:
# export as csv?
if export_as_csv == True:
    df.to_csv('./'+cyclone_name+'_canopycover_damage_with_windspeed_area_km2.csv')
else:
    pass